<a href="https://colab.research.google.com/github/bilnazir/stockmarketprediction/blob/LSTM_Google/fyp_LSTM_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=4b93feb615df5b17a3fdc4da9ed621ac368710e93d57609e991c48176955f4e8
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = 'GOOG', start = '2012-03-11',end = '2022-07-10')
data.head(10)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-03-12,14.943986,15.118333,14.925555,15.072255,15.055123,67010233
2012-03-13,15.161919,15.388570,15.082218,15.386826,15.369336,90168712
2012-03-14,15.317586,15.511360,15.279230,15.342244,15.324805,117916329
2012-03-15,15.357437,15.529292,15.313352,15.470264,15.452680,97769094
2012-03-16,15.464286,15.589317,15.443364,15.567649,15.549953,122477361
2012-03-19,15.519828,15.872257,15.473003,15.790314,15.772366,87237768
2012-03-20,15.714100,15.842120,15.623190,15.778110,15.760175,61850967
2012-03-21,15.806005,16.124311,15.753701,15.939754,15.921636,99154267
2012-03-22,15.902892,16.159430,15.716092,16.090937,16.072647,96769361


In [3]:
# Adding indicators
data['RSI']=ta.rsi(data.Close, length=15)
data['EMAF']=ta.ema(data.Close, length=20)
data['EMAM']=ta.ema(data.Close, length=100)
data['EMAS']=ta.ema(data.Close, length=150)

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)

data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

<ipython-input-3-bcaa738bef9c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]


In [5]:
data_set = data.iloc[:, 0:11]
pd.set_option('display.max_columns', None)

data_set.head(20)

,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
0,18.752213,18.891689,18.687206,18.695570,63.114599,18.472997,16.750024,15.845736,-0.197922,0,18.528139
1,18.726061,18.801277,18.533033,18.528139,58.291437,18.480257,16.785651,15.881544,-0.044888,0,18.434347
2,18.479235,18.526308,18.199286,18.434347,55.735068,18.477882,16.818714,15.915634,0.092741,1,18.526896
3,18.434155,18.605015,18.342747,18.526896,57.696439,18.484558,16.852957,15.950499,0.266031,1,18.795330
4,18.529299,18.837891,18.437391,18.795330,62.816630,18.516192,16.891843,15.988463,-1.527525,0,17.290442
5,18.817966,18.914602,16.836891,17.290442,36.373213,18.401328,16.900126,16.005968,-0.611832,0,16.961798
6,17.573629,17.601524,16.737265,16.961798,33.111772,18.266068,16.901729,16.018884,-0.077494,0,16.884180
7,16.961674,17.051836,16.679979,16.884180,32.377137,18.136290,16.901762,16.030599,0.188459,1,16.925974
8,16.737514,17.119083,16.737265,16.925974,33.231748,18.022856,16.902623,16.042713,-0.255787,0,16.850096
9,17.105883,17.110865,16.818708,16.850096,32.434406,17.912991,16.901963,16.053661,-0.074978,0,16.861540


In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.01961177 0.01934069 0.02123572 ... 0.45654945 0.         0.0180559 ]
 [0.01941916 0.01867509 0.02008558 ... 0.46732147 0.         0.01735823]
 [0.01760124 0.01665081 0.01759579 ... 0.47700908 1.         0.01804665]
 ...
 [0.6733618  0.71990078 0.67445715 ... 0.54020474 1.         0.73626929]
 [0.72180625 0.73682861 0.71922513 ... 0.69307994 1.         0.76667645]
 [0.73591794 0.76266871 0.74372024 ... 0.64518171 1.         0.77308485]]


In [7]:
# multiple feature from data provided to the model
X = []

backcandles = 30
print(data_set_scaled.shape[0])
for j in range(8):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
print(X)
print(X.shape)
print(y)
print(y.shape)

2449
[[[1.96117689e-02 1.93406886e-02 2.12357171e-02 ... 1.33156042e-02
   0.00000000e+00 0.00000000e+00]
  [1.94191573e-02 1.86750882e-02 2.00855754e-02 ... 1.33712291e-02
   2.85163557e-04 2.95199332e-04]
  [1.76012387e-02 1.66508076e-02 1.75957906e-02 ... 1.33530347e-02
   5.49797855e-04 5.76235737e-04]
  ...
  [4.21916472e-03 3.07675540e-03 4.53365508e-03 ... 3.47936822e-04
   5.51316076e-04 2.73534852e-03]
  [4.39893648e-03 3.11341804e-03 5.59274561e-03 ... 2.43070812e-04
   5.22467929e-04 2.78550651e-03]
  [3.75138165e-03 2.56335202e-03 4.27724908e-03 ... 2.42362799e-05
   4.67268690e-04 2.81645248e-03]]

 [[1.94191573e-02 1.86750882e-02 2.00855754e-02 ... 1.33712291e-02
   2.85163557e-04 2.95199332e-04]
  [1.76012387e-02 1.66508076e-02 1.75957906e-02 ... 1.33530347e-02
   5.49797855e-04 5.76235737e-04]
  [1.72692151e-02 1.72302346e-02 1.86660225e-02 ... 1.34041860e-02
   8.23876921e-04 8.63666809e-04]
  ...
  [4.39893648e-03 3.11341804e-03 5.59274561e-03 ... 2.43070812e-04
   5.

In [8]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

1935
(1935, 30, 8)
(484, 30, 8)
(1935, 1)
(484, 1)
[[0.00675268]
 [0.00827385]
 [0.009473  ]
 ...
 [0.42799209]
 [0.42447026]
 [0.42768002]]
